In [1]:
import sqlalchemy as db
from sqlalchemy import create_engine
from sqlalchemy import delete

In [2]:
engine = db.create_engine('sqlite:///C:\\Users\\peter\\Documents\\Projekte\\Zähler\\tc_all_new.db')

In [3]:
conn = engine.connect()

In [4]:
import pandas as pd

In [5]:
pd.options.display.max_rows=2000

### Erst einmal alle Tabellen auflisten

In [6]:
qt = " SELECT name FROM sqlite_master WHERE type='table' "

In [7]:
dftab = pd.read_sql_query(qt, conn)
dftab.head(8)

,name
0,tcsites
1,tcdata_CH_0529_05
2,tcdata_CH_0529_04
3,tcdata_CH_0529_03
4,tcdata_CH_0529_02
5,tcdata_CH_0529_01
6,tcdata_CH_0243_03
7,tcdata_CH_0243_04


In [8]:
tablist = dftab['name']

In [9]:
tablist = tablist[1:]

In [10]:
tablist

1     tcdata_CH_0529_05
2     tcdata_CH_0529_04
3     tcdata_CH_0529_03
4     tcdata_CH_0529_02
5     tcdata_CH_0529_01
6     tcdata_CH_0243_03
7     tcdata_CH_0243_04
8     tcdata_CH_0243_01
9     tcdata_CH_0243_02
10    tcdata_CH_0084_01
11    tcdata_CH_0084_03
12    tcdata_CH_0084_02
13    tcdata_CH_0084_04
14    tcdata_CH_0601_01
15    tcdata_CH_0601_02
16    tcdata_CH_0601_03
17    tcdata_CH_0601_04
18    tcdata_CH_0150_01
19    tcdata_CH_0150_02
20    tcdata_CH_0240_01
21    tcdata_CH_0240_02
22    tcdata_CH_0240_03
23    tcdata_CH_0240_04
24    tcdata_CH_0240_05
25    tcdata_CH_0240_06
26    tcdata_CH_0240_07
27    tcdata_CH_0240_08
28    tcdata_CH_0602_02
29    tcdata_CH_0602_03
30    tcdata_CH_0602_04
31    tcdata_CH_0602_01
32    tcdata_CH_0605_07
33    tcdata_CH_0605_08
34    tcdata_CH_0605_05
35    tcdata_CH_0605_06
36    tcdata_CH_0605_03
37    tcdata_CH_0605_04
38    tcdata_CH_0605_01
39    tcdata_CH_0605_02
40    tcdata_CH_0020_08
41    tcdata_CH_0020_02
42    tcdata_CH_

### OK, das ist die Liste aller Tabellen, die als String vorliegen!

---

### Und jetzt die Zeiträume feststellen, an denen es viele Multiple gibt ||

In [11]:
query ='''
SELECT 
    rowid as id, time as TIME, COUNT(time) as Anzahl
    FROM tcdata_CH_0084_04
    GROUP BY time
    HAVING COUNT() > 1
 '''

In [12]:
df = pd.read_sql_query(query, conn)
df.head(29)

,id,TIME,Anzahl
0,224233,2022-09-05 08:53:00+02:00,2
1,224501,2022-09-05 18:53:00+02:00,2
2,224812,2022-09-06 08:53:00+02:00,2
3,225076,2022-09-06 18:53:00+02:00,2
4,225386,2022-09-07 08:53:00+02:00,2
5,225653,2022-09-07 18:53:00+02:00,2
6,225963,2022-09-08 08:53:00+02:00,2
7,226228,2022-09-08 18:53:00+02:00,2
8,226538,2022-09-09 08:53:00+02:00,2
9,226804,2022-09-09 18:53:00+02:00,2


### ...also, die Zeilennummern (rowid) feststellen und die restlichen Multiplen löschen!

In [13]:
lst=df['id']

In [14]:
lst[0:8]

0    224233
1    224501
2    224812
3    225076
4    225386
5    225653
6    225963
7    226228
Name: id, dtype: int64

for t in lst:
    dq = "DELETE FROM tcdata_CH_0287_01 WHERE rowid = " + str(t)
    conn.execute(dq)

### ... unten zwei Abfragen, bei denen ich sehen kann, ob der einfache Count und der Count(distinct) gleich sind!

In [100]:
q_c ='''
SELECT 
    COUNT(DISTINCT time)
    FROM tcdata_CH_0287_02
 '''

In [101]:
res = conn.execute(q_c)
res.fetchone()

(169311,)

In [102]:
q_cd ='''
SELECT 
    COUNT()
    FROM tcdata_CH_0287_02
 '''

In [103]:
res = conn.execute(q_cd)
res.fetchone()

(171061,)

---

#### Ok, aber mit folgender Idee geht das Löschen mit einem Mal: Man stellt pro Zeitpunkt die Reihe mit der (zB) kleinsten **rowid** fest und löscht dann den Rest!

In [32]:
query ='''
SELECT 
    min(rowid) as id, time as TIME, COUNT(time) as Anzahl
    FROM tcdata_CH_0287_02
    GROUP BY time
    ORDER BY Anzahl DESC
 '''

In [34]:
df = pd.read_sql_query(query, conn)
df.head(8)

,id,TIME,Anzahl
0,204844,2022-08-31 05:48:00+02:00,1
1,204843,2022-08-31 05:43:00+02:00,1
2,204842,2022-08-31 05:38:00+02:00,1
3,204841,2022-08-31 05:33:00+02:00,1
4,204840,2022-08-31 05:28:00+02:00,1
5,204839,2022-08-31 05:23:00+02:00,1
6,204838,2022-08-31 05:18:00+02:00,1
7,204837,2022-08-31 05:13:00+02:00,1


#### ... und die ID im Einsatz...

In [20]:
d_query ='''
DELETE
    FROM tcdata_CH_0287_02
    WHERE rowid NOT IN
        (
        SELECT min(rowid)        
        FROM tcdata_CH_0287_02
        GROUP BY time
        )
 '''

In [21]:
conn.execute(d_query)

#### Und vor dem Loop noch eine Abfrage nach NULLs. Und Löschung der NULLs!

In [16]:
query ='''
SELECT 
    rowid as id, time as TIME, vfr11 as FLOW
    FROM tcdata_CH_0274_05
    WHERE vfr11 IS NULL
 '''

In [18]:
df = pd.read_sql_query(query, conn)
df.head(99)

,id,TIME,FLOW
0,221502,2022-08-31 16:54:00+02:00,None
1,221560,2022-08-31 17:52:00+02:00,None
2,221629,2022-08-31 19:28:00+02:00,None
3,222029,2022-09-01 16:04:00+02:00,None
4,222162,2022-09-01 18:18:00+02:00,None
5,222716,2022-09-02 17:56:00+02:00,None
6,222721,2022-09-02 18:01:00+02:00,None
7,223111,2022-09-03 10:58:00+02:00,None
8,223274,2022-09-03 17:38:00+02:00,None
9,223694,2022-09-04 11:34:00+02:00,None


### Bem. Den Datensatz zu 274_06 habe ich durch falsches Löschen platt gemacht!

#### Also, zum Abschluss in einem Loop alle Tabellen durchlaufen. Vermutlich muss ich die Query auf einer einzelnen Zeile schreiben

In [28]:
for tab in tablist:
    dq = "DELETE FROM " + tab + " WHERE rowid NOT IN (SELECT min(rowid) FROM " + tab + " GROUP BY time)"
    conn.execute(dq)

In [29]:
for tab in tablist:
    dq = "DELETE FROM " + tab + " WHERE vfr11 IS NULL"
    conn.execute(dq)

In [30]:
for tab in tablist:
    cq = "SELECT COUNT(DISTINCT time), COUNT() FROM " + tab 
    res = conn.execute(cq)
    num = res.fetchone()
    print(num)

(221101, 221101)
(191372, 191372)
(220152, 220152)
(166651, 166651)
(221181, 221181)
(221365, 221365)
(221416, 221416)
(221449, 221449)
(221159, 221159)
(221385, 221385)
(221385, 221385)
(221385, 221385)
(221385, 221385)
(221097, 221097)
(221093, 221093)
(221568, 221568)
(220813, 220813)
(220276, 220276)
(220224, 220224)
(221585, 221585)
(221599, 221599)
(221531, 221531)
(221546, 221546)
(221535, 221535)
(221528, 221528)
(221577, 221577)
(221599, 221599)
(220319, 220319)
(220319, 220319)
(220319, 220319)
(220319, 220319)
(221346, 221346)
(221024, 221024)
(221511, 221511)
(221583, 221583)
(221010, 221010)
(221595, 221595)
(221551, 221551)
(221115, 221115)
(221599, 221599)
(221557, 221557)
(221525, 221525)
(221599, 221599)
(221528, 221528)
(221581, 221581)
(221568, 221568)
(221543, 221543)
(203360, 203360)
(221291, 221291)
(220941, 220941)
(220906, 220906)
(16282, 16282)
(16282, 16282)
(16282, 16282)
(16282, 16282)
(16282, 16282)
(16282, 16282)
(16282, 16282)
(220252, 220252)
(2853, 2853

#### Ok, mir ist noch aufgefallen, dass ich ggf. Ausreisser bei Flow und speed habe!

In [20]:
outlier_flow1 = []

for tab in tablist:
    outlier_flow1.append(tab[10:])
    cq = "SELECT COUNT(time), MAX(vfr11) FROM " + tab + " WHERE vfr11 > 4200" 
    res = conn.execute(cq)
    num = res.fetchone()
    outlier_flow1.append(num)

In [21]:
print(outlier_flow1)

['0529_05', (0, None), '0529_04', (0, None), '0529_03', (0, None), '0529_02', (0, None), '0529_01', (0, None), '0243_03', (0, None), '0243_04', (0, None), '0243_01', (0, None), '0243_02', (0, None), '0084_01', (0, None), '0084_03', (0, None), '0084_02', (0, None), '0084_04', (0, None), '0601_01', (0, None), '0601_02', (0, None), '0601_03', (0, None), '0601_04', (0, None), '0150_01', (0, None), '0150_02', (0, None), '0240_01', (0, None), '0240_02', (0, None), '0240_03', (0, None), '0240_04', (0, None), '0240_05', (0, None), '0240_06', (0, None), '0240_07', (0, None), '0240_08', (0, None), '0602_02', (0, None), '0602_03', (0, None), '0602_04', (0, None), '0602_01', (0, None), '0605_07', (0, None), '0605_08', (0, None), '0605_05', (0, None), '0605_06', (0, None), '0605_03', (0, None), '0605_04', (0, None), '0605_01', (0, None), '0605_02', (0, None), '0020_08', (0, None), '0020_02', (0, None), '0020_03', (0, None), '0020_01', (0, None), '0020_06', (0, None), '0020_07', (0, None), '0020_04'

In [22]:
outlier_flow2 = []

for tab in tablist:
    outlier_flow1.append(tab[10:])
    cq = "SELECT COUNT(time), MAX(vfr21) FROM " + tab + " WHERE vfr21 > 1800" 
    res = conn.execute(cq)
    num = res.fetchone()
    outlier_flow2.append(num)
    outlier_flow2

In [23]:
print(outlier_flow2)

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (1, 1920.0), (0, None), (0, None), (0, None), (3, 2280.0), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]


In [24]:
outlier_speed1 = []

for tab in tablist:
    outlier_speed1.append(tab[10:])
    cq = "SELECT COUNT(time), MAX(s12) FROM " + tab + " WHERE s12 > 150" 
    res = conn.execute(cq)
    num = res.fetchone()
    outlier_speed1.append(num)
    outlier_speed1

In [25]:
print(outlier_speed1)

['0529_05', (4, 161.0), '0529_04', (40, 248.0), '0529_03', (1, 170.0), '0529_02', (1, 163.0), '0529_01', (6, 239.0), '0243_03', (41, 209.0), '0243_04', (5, 185.0), '0243_01', (155, 237.0), '0243_02', (402, 211.0), '0084_01', (0, None), '0084_03', (0, None), '0084_02', (0, None), '0084_04', (0, None), '0601_01', (0, None), '0601_02', (3, 168.0), '0601_03', (0, None), '0601_04', (0, None), '0150_01', (4, 210.0), '0150_02', (0, None), '0240_01', (0, None), '0240_02', (0, None), '0240_03', (0, None), '0240_04', (0, None), '0240_05', (0, None), '0240_06', (0, None), '0240_07', (0, None), '0240_08', (0, None), '0602_02', (0, None), '0602_03', (0, None), '0602_04', (0, None), '0602_01', (0, None), '0605_07', (1, 153.0), '0605_08', (1, 251.0), '0605_05', (0, None), '0605_06', (0, None), '0605_03', (0, None), '0605_04', (0, None), '0605_01', (0, None), '0605_02', (0, None), '0020_08', (0, None), '0020_02', (0, None), '0020_03', (0, None), '0020_01', (0, None), '0020_06', (0, None), '0020_07', (

In [26]:
outlier_speed2 = []

for tab in tablist:
    outlier_speed2.append(tab[10:])
    cq = "SELECT COUNT(time), MAX(s22) FROM " + tab + " WHERE s22 > 150" 
    res = conn.execute(cq)
    num = res.fetchone()
    outlier_speed2.append(num)
    outlier_speed2

In [27]:
print(outlier_speed2)

['0529_05', (26, 249.0), '0529_04', (22, 251.0), '0529_03', (6, 188.0), '0529_02', (33, 247.0), '0529_01', (41, 243.0), '0243_03', (19, 249.0), '0243_04', (23, 246.0), '0243_01', (34, 245.0), '0243_02', (54, 254.0), '0084_01', (0, None), '0084_03', (0, None), '0084_02', (0, None), '0084_04', (0, None), '0601_01', (25, 250.0), '0601_02', (39, 252.0), '0601_03', (6, 236.0), '0601_04', (56, 253.0), '0150_01', (1, 180.0), '0150_02', (1, 194.0), '0240_01', (0, None), '0240_02', (0, None), '0240_03', (0, None), '0240_04', (0, None), '0240_05', (0, None), '0240_06', (0, None), '0240_07', (0, None), '0240_08', (0, None), '0602_02', (0, None), '0602_03', (0, None), '0602_04', (0, None), '0602_01', (0, None), '0605_07', (38, 252.0), '0605_08', (38, 252.0), '0605_05', (25, 248.0), '0605_06', (12, 248.0), '0605_03', (69, 254.0), '0605_04', (5, 235.0), '0605_01', (13, 246.0), '0605_02', (33, 250.0), '0020_08', (0, None), '0020_02', (0, None), '0020_03', (0, None), '0020_01', (0, None), '0020_06', (

#### Ok, ich bereinige noch alle Rows mit Flows über 4200

In [165]:
for tab in tablist:
    dq = "DELETE FROM " + tab + " WHERE vfr11 > 4200"
    conn.execute(dq)

### ende, aus ... Stand 04092022